In [28]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import sys

In [30]:
filename = "alice_in_wonderland.txt"  # Update with your dataset
raw_text = open(filename, "r", encoding="utf-8").read().lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [31]:
X = []
y = []
for i in range(0, len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

# Reshape for LSTM Input
X = np.reshape(X, (X.shape[0], X.shape[1]))

In [37]:
model = Sequential()
model.add(LSTM(2512, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation="softmax"))

IndexError: tuple index out of range

In [34]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001))

In [35]:
filepath = "weights-improvement-{epoch:01d}-{loss:.4f}.hdf5"  # Store path and type
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

In [36]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20


TypeError: in user code:

    File "C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
    

In [ ]:
f_name = "weights-improvement-14-2.0773.hdf5"
model.load_weights(f_name)

In [ ]:
def generate_response(prompt, model, char_to_int, int_to_char, seq_len=100, n_vocab=0):
    # Preprocess input
    prompt = prompt.lower()
    input_pattern = [char_to_int[char] for char in prompt if char in char_to_int]
    
    # Pad or trim input to match sequence length
    while len(input_pattern) < seq_len:
        input_pattern.insert(0, 0)  # Pad with zero
    input_pattern = input_pattern[-seq_len:]
    # Generate response
    generated_text = ""
    for _ in range(200):  # Limit response length
        x = numpy.reshape(input_pattern, (1, seq_len, 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        generated_text += result
        
        # Update input pattern
        input_pattern.append(index)
        input_pattern = input_pattern[1:]
    
    return generated_text.strip()

In [ ]:
# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break
    response = generate_response(user_input, model, char_to_int, int_to_char, seq_len, n_vocab)
    print(f"Chatbot: {response}")